# Projeto 1 - Classificador Naive-Bayes com Tweets


## Objetivo

* Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa selecionealgumas mensagens disponíveis no Twitter que mencionam esse produto e classifique-as como"relevante" ou "irrelevante".

* Com isso, essa empresa deseja que mensagens relevantes, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.

* Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamentedo Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails, porexemplo. Esse classificador permite calcular qual a probabilidade de uma mensagem ser relevante dada as palavras em seu conteúdo.

* Para realizar o MVP (minimum viable product) do projeto, você precisa implementar umaversão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.

* Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Integrantes do Projeto

**Nome:** Wilgner Lopes Araujo

**Nome:** Fabricio Neri Lima

## Informações relevantes

* Esse arquivo representa a ultima etapa do projeto onde haverá todas as etapas de cleaning da base de dados e implantação do algoritmo de classificação

* O arquivo que pesquisa os tweets esta na pasta Arquivo Jupyter/Projeto1_Obtenção_dos_tweets nessa mesma raiz do repositório

* A base de dados se encontra na pasta dados/Valorant.xlsx

* A conta utiliza no twitter para obtenção de dados é @AnalyticsInsper

* O produto escolhido para o projeto é : **Valorant**

* Os criterios para a classificação dos tweets na primeira etapa do projeto foram: Os tweets que falavam de alguma forma positiva ou negativa sobre o jogo ou até mesmo que relacionavam o jogo a alguma emoção, foram classificados como relevantes. Os demais foram classificados como Irrelevantes.


# Começando

## Declarando as bibliotecas

In [1]:
import math
import numpy as np
import pandas as pd
import re

## Abrindo a base de dados e visualizando os primeiros 5 termos

In [2]:
dados = pd.read_excel('../dados/Valorant.xlsx')
dados.head()

,Treinamento,Etiquetas,Classificado
0,@caos__0 eu no valorant :v\nfon,2,Irrelevante
1,o cara q pra mim é o melhor player de valorant...,2,Irrelevante
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante
3,"deletei o valorant, agora meu pc volta a ligar...",1,Relevante
4,quem tiver pc baixa valorant nessa porr que eu...,1,Relevante


## Informações do DataFrame

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Treinamento   500 non-null    object
 1   Etiquetas     500 non-null    int64 
 2   Classificado  500 non-null    object
dtypes: int64(1), object(2)
memory usage: 11.8+ KB


## Classificando 

* A coluna Treinamento e Classificado são categoricas, pois ambas são variaveis qualitativas nominais

In [4]:
dados['Treinamento'] = dados['Treinamento'].astype('category')
dados['Classificado'] = dados['Classificado'].astype('category')

## Iniciando as coletas de probabilidades

### Analisando frequencia de Relevante e Irrelevante na base de dados

In [5]:
Probabilidade_total = dados.Classificado.value_counts(True)
Probabilidade_total

Irrelevante    0.542
Relevante      0.458
Name: Classificado, dtype: float64

## Declarando as variaveis de probabilidade

* Considere P(R) para probabilidade de ser Relevante
* Considere P(R_c) para probabilidade de ser Irrelevante

In [6]:
P_R_c = Probabilidade_total[0]
P_R = Probabilidade_total[1]

## Antes de prosseguir vamos limpar o codigo

* Será removido os seguintes simbolos: \n, :, ", ', (,), ., ;, :, ?, !, <, >, 

In [7]:
def limpa_texto(text):
    
    #import string
    punctuation = '[!-.:?;@_,<>/\)(#%$|\n]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

## Aplicando a função de limpeza na base de dados

In [8]:
dados['Treinamento'] = dados['Treinamento'].apply(limpa_texto)


In [9]:
# Checando se funcionou e se os emojis ainda funcionam
pd.set_option('max_colwidth', -1)
dados.head(30)

C:\Users\wilgn\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Treinamento,Etiquetas,Classificado
0,caos0 eu no valorant vfon,2,Irrelevante
1,o cara q pra mim é o melhor player de valorant do mundo joga osu,2,Irrelevante
2,mano algm upa meu passe no valorant na humildade,2,Irrelevante
3,deletei o valorant agora meu pc volta a ligar em 3 segundos,1,Relevante
4,quem tiver pc baixa valorant nessa porr que eu tô no ódio e vai ser o dia todo só isso,1,Relevante
5,voltei a jogar valorant so por causa do mata mata,1,Relevante
6,viciei em valorant esse é o tweet,1,Relevante
7,zncabral free fire fez mais sucesso que o valorant só no brasil no resto do mundo quase ngm joga isso,1,Relevante
8,alguem me explica o motivo das 12 do valorant menos a shorty atirar tão bem de longe tipo comparando com cs e fortnite não faz sentido os cara vai de bucky atacando vai se fuder,1,Relevante
9,rt leozjn o matchmaking do valorant n faz o minimo sentido,1,Relevante


## Separando a base de dados em Relevantes e Irrelevantes

In [10]:
filtro_relevante = dados.Classificado == 'Relevante'
filtro_irrelevante = dados.Classificado == 'Irrelevante'

dados_relevantes = dados.loc[filtro_relevante,:]
dados_irrelevantes = dados.loc[filtro_irrelevante,:]

In [11]:
dados_relevantes.tail(15)

,Treinamento,Etiquetas,Classificado
460,eu to fechando e abrindo o valorant pq simplesmente sem nada p fazer,1,Relevante
461,que toda tristeza seja convertida em skill de valorant amém,1,Relevante
462,quero alguém p jogar valorant cmg nesse caraleo,1,Relevante
464,2 dia sem jogar valorant e eu já fico completamente horrível meu deus,1,Relevante
465,crianças jogando valorant 120 da manha ai eu pergunto onde ta a mae,1,Relevante
466,jogar valorant é dar bala em radiante e imortal e não conseguir sair do platina pq jogo solo,1,Relevante
467,fds among fdp eu vou pegar diamante no valorant se fuder porra😐,1,Relevante
474,eu era apaixonada em ver as pessoas jogarem valorant mas agora não consigo nem olhar a tela do jogo aberta,1,Relevante
476,sempre q eu jogo normal gamezin no valorant sempre tem um cara mt baludo no outro time,1,Relevante
477,rt diogogmachadof valorant é um jogo de corno,1,Relevante


In [12]:
frase = 0
frase_quebrada = []

for i in range(len(dados_relevantes)):
    frase = dados_relevantes.iloc[i,0]
    frase_quebrada += frase.split()

palavras_quebradas_relevantes = pd.Series(frase_quebrada)

'''frase_quebrada_total = []
for i in range(len(dados_relevantes)):
    frase = dados_relevantes.iloc[i,0]
    frase_quebrada_total += frase.split()

palavras_quebradas_total = pd.Series(frase_quebrada_total)'''

palavras_quebradas_relevantes_relativa = palavras_quebradas_relevantes.value_counts(True)
palavras_quebradas_relevantes_relativa

valorant      0.072638
de            0.030945
eu            0.023453
no            0.022476
o             0.019870
                ...   
caraleo       0.000326
haven         0.000326
fé            0.000326
chat          0.000326
apaixonado    0.000326
Length: 1079, dtype: float64

In [13]:
frase = 0
frase_quebrada = list()

for i in range(len(dados_relevantes)):
    frase = dados_relevantes.iloc[i,0]
    frase_quebrada.append(frase)
    
frase_quebrada

lista_de_lista_de_tweets = list()

for palavra in range(len(frase_quebrada)):
    palavra = frase_quebrada[palavra].split()
    lista_de_lista_de_tweets.append(palavra)
    
lista_de_lista_de_tweets[0][0]
prob_tweet_dado_relevante = 1
lista_proba = list()
valida = list()
x = 0
x_0 = 0

for trecho in range(len(lista_de_lista_de_tweets)):
#     print(trecho)
    
    for palavra in lista_de_lista_de_tweets[trecho]:
        
        
        
        print(lista_de_lista_de_tweets[trecho])
#         print(palavra)
        print(palavras_quebradas_relevantes_relativa[palavra])
        prob_tweet_dado_relevante *= palavras_quebradas_relevantes_relativa[palavra]
        print(prob_tweet_dado_relevante)
        
#     print('CORTA')
    lista_proba.append(prob_tweet_dado_relevante)
    prob_tweet_dado_relevante = 1
    if trecho >= 0:
            break
    
    
print(lista_proba)

['deletei', 'o', 'valorant', 'agora', 'meu', 'pc', 'volta', 'a', 'ligar', 'em', '3', 'segundos']
0.0003257328990228013
0.0003257328990228013
['deletei', 'o', 'valorant', 'agora', 'meu', 'pc', 'volta', 'a', 'ligar', 'em', '3', 'segundos']
0.01986970684039088
6.472217211853707e-06
['deletei', 'o', 'valorant', 'agora', 'meu', 'pc', 'volta', 'a', 'ligar', 'em', '3', 'segundos']
0.07263843648208469
4.701317388414908e-07
['deletei', 'o', 'valorant', 'agora', 'meu', 'pc', 'volta', 'a', 'ligar', 'em', '3', 'segundos']
0.001954397394136808
9.18824245292816e-10
['deletei', 'o', 'valorant', 'agora', 'meu', 'pc', 'volta', 'a', 'ligar', 'em', '3', 'segundos']
0.008143322475570033
7.482282127791661e-12
['deletei', 'o', 'valorant', 'agora', 'meu', 'pc', 'volta', 'a', 'ligar', 'em', '3', 'segundos']
0.0035830618892508143
2.6809479936712792e-14
['deletei', 'o', 'valorant', 'agora', 'meu', 'pc', 'volta', 'a', 'ligar', 'em', '3', 'segundos']
0.000977198697068404
2.6198188863237256e-17
['deletei', 'o', 'v

In [ ]:
for trecho in range(len(lista_de_lista_de_tweets)):
    for palavra in lista_de_lista_de_tweets[trecho]:
        
        if lista_de_lista_de_tweets[trecho] not in  valida:
            print(lista_de_lista_de_tweets[trecho])
            print(palavra)
            prob_tweet_dado_relevante *= palavras_quebradas_relevantes_relativa[palavra]
            lista_proba.append(prob_tweet_dado_relevante)
        else:
            prob_tweet_dado_relevante = 1
            pass
        valida.append(lista_de_lista_de_tweets[trecho])

## 
